In [ ]:
# 作业 （1和2有关系，非独立）
# 1. 搭建/使用预训练模型进行cifar10数据集分类任务： 之前没学卷积神经网络的时候，在cifar10数据集上看预测分类的表现。  现在可以换成用CNN（可以是自己搭的残差网络 重新训练，也可以加载预训练的模型，然后做这个分类问题做迁移学习）。
# 
# 2. Flask应用开发：创建flask应用，实现图片上传，能够利用上一题搭建/预训练的模型进行图片分类
# 
# 3. 前端界面可以显示预测结果 和 预测的概率。


In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

def load_and_preprocess_cifar10():
    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
    x_train = train_images / 255.0
    x_test = test_images / 255.0
    y_train = tf.keras.utils.to_categorical(train_labels, 10)
    y_test = tf.keras.utils.to_categorical(test_labels, 10)
    
    return (x_train, y_train), (x_test, y_test)

In [6]:
def create_resnet_model():
    input = tf.keras.Input(shape=(32, 32, 3))
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    shortcut = tf.keras.layers.Conv2D(64, 1, strides=2)(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)
    x = tf.keras.layers.add([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)
    shortcut = tf.keras.layers.Conv2D(128, 1, strides=2)(x)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)
    x = tf.keras.layers.add([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    output = tf.keras.layers.Dense(10, activation='softmax')(x)
    model = tf.keras.Model(input, output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
(x_train, y_train), (x_test, y_test) = load_and_preprocess_cifar10()
    
# 创建模型
model = create_resnet_model()

# 训练模型
model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=10,
                    validation_data=(x_test, y_test))


Epoch 1/10
1563/1563 [==============================] - 109s 69ms/step - loss: 1.5626 - accuracy: 0.4171 - val_loss: 1.2494 - val_accuracy: 0.5427
Epoch 2/10
1563/1563 [==============================] - 99s 63ms/step - loss: 1.0758 - accuracy: 0.6141 - val_loss: 0.9178 - val_accuracy: 0.6718
Epoch 3/10
1563/1563 [==============================] - 93s 60ms/step - loss: 0.8610 - accuracy: 0.6956 - val_loss: 0.8339 - val_accuracy: 0.7082
Epoch 4/10
1563/1563 [==============================] - 93s 60ms/step - loss: 0.7158 - accuracy: 0.7477 - val_loss: 0.9208 - val_accuracy: 0.6822
Epoch 5/10
1563/1563 [==============================] - 97s 62ms/step - loss: 0.6166 - accuracy: 0.7847 - val_loss: 0.6800 - val_accuracy: 0.7643
Epoch 6/10
1563/1563 [==============================] - 98s 63ms/step - loss: 0.5342 - accuracy: 0.8123 - val_loss: 0.7086 - val_accuracy: 0.7634
Epoch 7/10
1563/1563 [==============================] - 97s 62ms/step - loss: 0.4659 - accuracy: 0.8356 - val_loss: 0.6839 

In [8]:
model.save('cifar10_model.h5')

C:\Users\31752\.conda\envs\homl3\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
